<pre>
<b>
COMP-6721 | 2021-Winter
Project | Part-1

Pravesh Gupta | 40152506
Vikramjeet Singh | 
Manjot Kaur Dherdi | 
</b>
</pre>

# 1. Data Loading

## 1.1. Installing required python modules

In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


### 1.2. Dataset Description


Required Dataset Folder Heirarchy:
<pre>
{dataset_folder}
--images
----mask
-------{mask images}
----no_mask
-------{no mask images}
----not_person
-------{not person images}
</pre>

### 1.3. Defining Pytorch dataset Representation and data transaformers

In [6]:
from torchvision.datasets import VisionDataset
import pandas as pd
import os
from skimage import io as sk_io, transform as sk_transform
import numpy as np
from sklearn.utils import shuffle
from PIL import Image
    
class Rescale(object):
    def __init__(self, output_size, debug=False, export_path=None):
        assert isinstance(output_size, (int))
        self.output_size = output_size

    def __call__(self, img_data):
        img_arr = np.array(img_data)
        scale_factor = float(self.output_size)/img_arr.shape[0]
        img_arr = (sk_transform.rescale(img_arr, (scale_factor, scale_factor, 1))*255).astype(np.uint8)
        
        new_w = img_arr.shape[1]
        
        # Clipping or filling
        if new_w>self.output_size:
            mid = new_w//2
            new_w_start = mid-self.output_size//2
            new_w_end = mid+self.output_size//2
            
            if (new_w_end-new_w_start)<self.output_size:
                new_w_end += (self.output_size-(new_w_end-new_w_start))
            elif (new_w_end-new_w_start)>self.output_size:
                new_w_end -= ((new_w_end-new_w_start)-self.output_size)
            img_arr = img_arr[:, new_w_start:new_w_end]
        elif new_w<self.output_size:
            mid = new_w//2
            new_w_start = self.output_size//2-mid
            new_w_end = new_w_start+new_w
            filled_img_arr = np.zeros((self.output_size, self.output_size, img_arr.shape[2]), dtype=np.uint8)
            filled_img_arr[:, new_w_start:new_w_end] = img_arr[:, :]
            img_arr = filled_img_arr
        return Image.fromarray(img_arr)
    
class ConvertToType(object):
    def __init__(self, conversion_type, debug=False):
        self.conversion_type = conversion_type
        self.debug = debug

    def __call__(self, img):
        return img.type(self.conversion_type)

### 1.4. Creating dataset and data loaders for train and test

In [95]:
def get_train_test_indices(dataset_targets):
    test_indices_map = {}
    for class_name in dataset.class_to_idx:
        label = dataset.class_to_idx[class_name]
        test_indices_map[label] = {'indices': np.array([], dtype=np.int32), 'count': 0}

    train_indices_map = {}
    for class_name in dataset.class_to_idx:
        label = dataset.class_to_idx[class_name]
        train_indices_map[label] = {'indices': np.array([], dtype=np.int32), 'count': 0}

    targets = np.array(dataset.targets, dtype=np.int32)
    target_indices = np.where(targets!=None)[0]
    np.random.shuffle(target_indices)

    for i in target_indices:
        label = dataset.targets[i]
        if test_indices_map[label]['count']<100:
            test_indices_map[label]['indices'] = np.append(test_indices_map[label]['indices'], i)
            test_indices_map[label]['count']+=1
        elif train_indices_map[label]['count']<350:
            train_indices_map[label]['indices'] = np.append(train_indices_map[label]['indices'], i)
            train_indices_map[label]['count']+=1

    test_indices = np.array([], dtype=np.int32)
    train_indices = np.array([], dtype=np.int32)

    for class_name in dataset.class_to_idx:
        label = dataset.class_to_idx[class_name]
        label_test_indices = test_indices_map[label]['indices']
        test_indices = np.append(test_indices, label_test_indices)

        label_train_indices = train_indices_map[label]['indices']
        train_indices = np.append(train_indices, label_train_indices)
    return train_indices, test_indices

In [96]:
from torchvision.transforms import ToTensor, Compose, Normalize
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

data_transform = Compose([
    Rescale(512)
    ,ToTensor()
    ,Normalize(mean=torch.zeros((3)),std=torch.ones((1)))
])

dataset = ImageFolder(
    root='D:/pravesh/Concordia/2021-Winter/COMP-6721-Intro_To_AI/project/1/dataset/images/'
    ,transform=data_transform
)
print(f'Dataset: size: {len(dataset)}, class labels: {dataset.class_to_idx}')

train_indices, test_indices = get_train_test_indices(dataset.targets)
    
train_dataset = Subset(dataset, train_indices)
print(f'Train dataset: size: {len(train_dataset)}')

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
for data, labels in train_dataloader:
    print(f'Train X | y batch shapes : {data.shape, labels.shape}')
    break

test_dataset = Subset(dataset, test_indices)
print(f'Test dataset: size: {len(test_dataset)}')

test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)
for data, labels in test_dataloader:
    print(f'Test X | y batch shapes : {data.shape, labels.shape}')
    break

Dataset: size: 3091, class labels: {'mask': 0, 'no_mask': 1, 'not_person': 2}
Train dataset: size: 1050
Train X | y batch shapes : (torch.Size([10, 3, 512, 512]), torch.Size([10]))
Test dataset: size: 300
Test X | y batch shapes : (torch.Size([10, 3, 512, 512]), torch.Size([10]))


# 2. Data Modelling

In [97]:
from torch.nn import Module, Conv2d, MaxPool2d, Linear, ReLU

class ProjectModel(Module):
    def __init__(self):
        super(ProjectModel, self).__init__()
        self.conv1 = Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=2)
        self.pool1 = MaxPool2d(2)
        self.conv2 = Conv2d(in_channels=16, out_channels = 32, kernel_size=3)
        self.linear = Linear(in_features = 32*125*125, out_features=3)
        
    def forward(self, X):
        relu = ReLU(inplace=False)
        X = self.conv1(X)
        X = relu(X)
        X = self.pool1(X)
        X = self.conv2(X)
        X = relu(X)
        X = torch.flatten(X, 1)
        X = self.linear(X)
        return X

### 2.1. Integrity Test

In [98]:
from sklearn import metrics

# Model Unit test

for data, labels in train_dataloader:
    net = ProjectModel()
    outputs = net(data)
    _, y_pred = torch.max(outputs.data, 1)
    acc = metrics.accuracy_score(labels, y_pred, normalize=True)
    break
print('Model test passed.')

Model test passed.


### 2.2. Training

In [15]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
import time

num_epoch = 5
lr = 0.01
momentum = 0.9

net = ProjectModel()
loss_evaluater = CrossEntropyLoss()
optimizer = SGD(net.parameters(), lr=lr, momentum=momentum)

t = time.time()
for i in range(num_epoch):
    train_acc_sum = 0.0
    batch_count = 0
    t_epoch = time.time()
    for data, labels in train_dataloader:
        X, y = data, labels
        optimizer.zero_grad()
        output = net(X)
        loss = loss_evaluater(output, y)
        loss.backward()
        optimizer.step()
        _, y_pred = torch.max(output.data, 1)
        acc = metrics.accuracy_score(y, y_pred)
        batch_count+=1
        train_acc_sum+=acc
    epoch_acc = (train_acc_sum/batch_count)
    print(f'Epoch {i+1}: [time:{time.time()-t_epoch} seconds, Accuracy: {epoch_acc*100}]')
print(f'Training time: {time.time()-t} seconds')
print('=============================================')

t = time.time()
with torch.no_grad():
    acc_sum = 0.0
    batch_count = 0
    for data, labels in train_dataloader:
        X, y = data, labels
        output = net(X)
        _, y_pred = torch.max(output.data, 1)
        acc = metrics.accuracy_score(y, y_pred)
        acc_sum+=acc
        batch_count+=1
    train_acc = acc_sum/batch_count
    print(f'Training accuracy: {train_acc*100}')

print(f'Trainig evaluation time: {time.time()-t} seconds')

Epoch 1: [time:826.873375415802 seconds, Accuracy: 31.714285714285722
Epoch 2: [time:697.1080617904663 seconds, Accuracy: 46.19047619047618
Epoch 3: [time:683.4369280338287 seconds, Accuracy: 56.5714285714286
Epoch 4: [time:679.9524466991425 seconds, Accuracy: 75.52380952380953
Epoch 5: [time:686.3944699764252 seconds, Accuracy: 79.14285714285715
Training time: 3573.7712635993958
Training accuracy: 89.80952380952384
Trainig evaluation time: 674.6690237522125


### 2.3. Testing

In [16]:
acc_sum = 0.0
batch_count = 0
for data, labels in test_dataloader:
    X, y = data, labels
    output = net(X)
    _, y_pred = torch.max(output.data, 1)
    acc = metrics.accuracy_score(y, y_pred)
    acc_sum+=acc
    batch_count+=1
test_acc = acc_sum/batch_count
print(f'Test accuracy: {test_acc*100}')

Test accuracy: 45.0


# 3. Model Evaluation

### 3.1. Accuracy